In [1]:
import keras
import tensorflow
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


Matplotlib is building the font cache; this may take a moment.


In [15]:
! pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# Access the features and targets
X = wine_quality.data.features
y = wine_quality.data.targets

# Print metadata (if available)
if hasattr(wine_quality, 'metadata'):
    print("Metadata:")
    for key, value in wine_quality.metadata.items():
        print(f"{key}: {value}")
else:
    print("Metadata not available")

# Print variable information (if available)
if hasattr(wine_quality, 'variables'):
    print("\nVariable Information:")
    for var, info in wine_quality.variables.items():
        print(f"{var}: {info}")
else:
    print("Variable information not available")

Metadata:
uci_id: 186
name: Wine Quality
repository_url: https://archive.ics.uci.edu/dataset/186/wine+quality
data_url: https://archive.ics.uci.edu/static/public/186/data.csv
abstract: Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).
area: Business
tasks: ['Classification', 'Regression']
characteristics: ['Multivariate']
num_instances: 4898
num_features: 11
feature_types: ['Real']
demographics: []
target_col: ['quality']
index_col: None
has_missing_values: no
missing_values_symbol: None
year_of_dataset_creation: 2009
last_updated: Wed Nov 15 2023
dataset_doi: 10.24432/C56S3T
creators: ['Paulo Cortez', 'A. Cerdeira', 'F. Almeida', 'T. Matos', 'J. Reis']
intro_paper: {'ID': 252, 'type': 'NATIVE', 'title': 'Modeling wine preferences by data mining from physicochemical properties', 'authors': 'P. Cort

In [17]:
data = pd.concat([X, y], axis=1)
data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [19]:
train, test = train_test_split(data, test_size=0.3,random_state=42 )

In [24]:
train_X = train.drop(['quality'],axis=1).values
train_Y = train[['quality']].values.ravel()

test_X = test.drop(['quality'],axis=1).values
test_Y = test[['quality']].values.ravel()

train_x, valid_x, train_y, valid_y = train_test_split(train_X,train_Y, test_size=0.30,random_state=42)

In [25]:
signature = infer_signature(train_x,train_y)

In [ ]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [ ]:
def objective(params):
    # MLflow will track the parameters and results for each run(epoch)
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_X,
        test_y=test_Y,
    )
    return result

In [31]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [32]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/50 ━━━━━━━━━━━━━━━━━━━━ 26s 542ms/step - loss: 34.0270 - root_mean_squared_error: 5.8333
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 31.0647 - root_mean_squared_error: 5.5717 - val_loss: 24.0906 - val_root_mean_squared_error: 4.9082

Epoch 2/3                                            

 1/50 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 24.7117 - root_mean_squared_error: 4.9711
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.6149 - root_mean_squared_error: 4.7535 - val_loss: 17.5671 - val_root_mean_squared_error: 4.1913

Epoch 3/3                                            

 1/50 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 17.5347 - root_mean_squared_error: 4.1874
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.2343 - root_mean_squared_error: 4.0283 - val_loss: 12.6590 - val_root_mean_squared_error: 3.5579

 1/22 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 13.5341 - root_mean_squared_error: 3.6789
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s

In [33]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/86fac301b8384ce9af1448b4114eafe7/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_X)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step


array([[2.9478798],
       [1.629839 ],
       [2.4232712],
       ...,
       [3.6639225],
       [4.1764517],
       [3.6916752]], dtype=float32)

In [34]:
# Load model as a PyFuncModel.
model_uri = 'runs:/86fac301b8384ce9af1448b4114eafe7/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_X))

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step


array([[2.9478798],
       [1.629839 ],
       [2.4232712],
       ...,
       [3.6639225],
       [4.1764517],
       [3.6916752]], dtype=float32)

In [35]:
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1733020610241, current_stage='None', description=None, last_updated_timestamp=1733020610241, name='wine-quality', run_id='86fac301b8384ce9af1448b4114eafe7', run_link=None, source='file:///Users/sathvik/Desktop/Langchain/DLwithMLFlow/mlruns/841483789191681239/86fac301b8384ce9af1448b4114eafe7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>